# Updated Labels from Stulberg Classification
## Extract and match Stulberg labels with existing classification metadata

This notebook:
- Loads Stulberg classification data from second sheet
- Parses image filenames to extract patient number, view, and side
- Filters for AP (anterior-posterior) view images only
- Matches with existing classification_metadata.xlsx
- Creates updated classification_metadata2.xlsx with matched entries only

## 1. Imports and Setup

In [ ]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

# Setup paths
BASE_DIR = Path('C:/FeatureEx')
STULBERG_FILE = BASE_DIR / 'Stulberg classification.xlsx'
METADATA_FILE = BASE_DIR / 'classification_metadata.xlsx'
OUTPUT_FILE = BASE_DIR / 'classification_metadata2.xlsx'

print(f"Base directory: {BASE_DIR}")
print(f"Input files:")
print(f"  Stulberg: {STULBERG_FILE}")
print(f"  Metadata: {METADATA_FILE}")
print(f"\nOutput file:")
print(f"  {OUTPUT_FILE}")

## 2. Load Stulberg Classification Data

In [ ]:
# Load Stulberg classification from second sheet
print("Loading Stulberg classification data...")

# First, let's check what sheets are available
xls = pd.ExcelFile(STULBERG_FILE)
print(f"\nAvailable sheets in Stulberg classification.xlsx:")
for i, sheet in enumerate(xls.sheet_names):
    print(f"  {i}: {sheet}")

# Load the second sheet (index 1)
stulberg_df = pd.read_excel(STULBERG_FILE, sheet_name=1)

print(f"\nStulberg data shape: {stulberg_df.shape}")
print(f"\nFirst few rows:")
print(stulberg_df.head(10))

print(f"\nColumn names:")
print(stulberg_df.columns.tolist())

print(f"\nData types:")
print(stulberg_df.dtypes)

## 3. Parse Image Filenames to Extract Patient, View, and Side

In [ ]:
# Extract information from image filenames
print("Parsing image filenames to extract patient number, view, and side...\n")

def parse_image_filename(filename):
    """
    Parse image filename in format: Patient_X_VIEW_..._SIDE.bmp
    Example: Patient_1_AP_..._R.bmp
    
    Returns: {'patient_number': str, 'view': str, 'side': str}
    """
    if pd.isna(filename) or not isinstance(filename, str):
        return {'patient_number': None, 'view': None, 'side': None}
    
    filename = filename.strip()
    
    # Pattern: Patient_X_VIEW_..._SIDE.bmp
    # We need to extract:
    # 1. Patient number: after 'Patient_' up to the next underscore
    # 2. View: the next component (AP, LAT, PA, etc.)
    # 3. Side: the last character before .bmp extension (L or R)
    
    try:
        # Remove .bmp extension
        name_no_ext = filename.replace('.bmp', '').replace('.BMP', '')
        
        # Split by underscore
        parts = name_no_ext.split('_')
        
        if len(parts) >= 3:
            # parts[0] = 'Patient'
            # parts[1] = patient number
            # parts[2] = view (AP, LAT, PA, etc.)
            # parts[-1] = side (L or R)
            
            patient_num = parts[1]  # e.g., '1', '1008', etc.
            view = parts[2]         # e.g., 'AP', 'LAT', 'PA'
            side = parts[-1]        # e.g., 'L', 'R'
            
            return {
                'patient_number': patient_num,
                'view': view,
                'side': side
            }
    except Exception as e:
        print(f"Error parsing '{filename}': {e}")
    
    return {'patient_number': None, 'view': None, 'side': None}


# Apply parsing to first column (image filename column)
first_col = stulberg_df.iloc[:, 0]  # Get the first column
print(f"Sample filenames from first column:")
print(first_col.head(10).tolist())

# Parse all filenames
parsed_data = first_col.apply(parse_image_filename).apply(pd.Series)

print(f"\nParsed data (first 10 rows):")
print(parsed_data.head(10))

# Add parsed columns to dataframe
stulberg_df['patient_number'] = parsed_data['patient_number']
stulberg_df['view'] = parsed_data['view']
stulberg_df['side'] = parsed_data['side']

print(f"\nStulberg data with parsed columns:")
print(stulberg_df.head(10))

## 4. Filter for AP View Images Only

In [ ]:
# Filter for AP view images only
print("Filtering for AP view images only...\n")

print(f"View distribution in original data:")
print(stulberg_df['view'].value_counts())

# Filter for AP images
stulberg_ap_df = stulberg_df[stulberg_df['view'] == 'AP'].copy()

print(f"\nAP images only:")
print(f"  Total rows: {len(stulberg_ap_df)}")
print(f"\nFirst few AP entries:")
print(stulberg_ap_df.head(10))

# Display relevant columns
print(f"\nRelevant columns:")
print(stulberg_ap_df[['patient_number', 'view', 'side', 'Alex']].head(15))

## 5. Load Classification Metadata

In [ ]:
# Load existing classification metadata
print("Loading classification_metadata.xlsx...\n")

metadata_df = pd.read_excel(METADATA_FILE, sheet_name='samples')

print(f"Metadata shape: {metadata_df.shape}")
print(f"\nFirst few rows:")
print(metadata_df.head(10))

print(f"\nColumn names:")
print(metadata_df.columns.tolist())

print(f"\nData types:")
print(metadata_df.dtypes)

print(f"\nFirst column (sample_id) sample values:")
print(metadata_df.iloc[:, 0].head(15).tolist())

## 6. Create Matching Key for Stulberg Data

In [ ]:
# Create a matching key in format: 'Patient XXXX Side' or 'Patient X Side'
print("Creating matching keys for AP images...\n")

def create_patient_side_key(row):
    """
    Create matching key in format: 'Patient XXXX Side'
    Example: 'Patient 1008 Right' or 'Patient 99 Left'
    """
    patient_num = row['patient_number']
    side = row['side']
    
    if pd.isna(patient_num) or pd.isna(side):
        return None
    
    # Convert side (L/R to Left/Right)
    side_full = 'Right' if side == 'R' else 'Left' if side == 'L' else None
    
    if side_full is None:
        return None
    
    return f"Patient {patient_num} {side_full}"

# Apply to AP filtered data
stulberg_ap_df['match_key'] = stulberg_ap_df.apply(create_patient_side_key, axis=1)

print(f"Sample match keys:")
print(stulberg_ap_df[['patient_number', 'side', 'match_key']].head(15))

print(f"\nMatch key statistics:")
print(f"  Total AP entries: {len(stulberg_ap_df)}")
print(f"  Valid match keys: {stulberg_ap_df['match_key'].notna().sum()}")
print(f"  Missing match keys: {stulberg_ap_df['match_key'].isna().sum()}")

## 7. Parse Classification Metadata Sample IDs

In [ ]:
# The first column of classification_metadata.xlsx contains sample_id
# We need to create a matching key from these
print("Extracting patient and side info from classification_metadata.xlsx...\n")

# The first column name
id_column = metadata_df.columns[0]
print(f"ID column: '{id_column}'")

# Sample values
print(f"\nSample ID values:")
print(metadata_df[id_column].unique()[:20])

# Create the match key directly from the sample_id
# It appears these are already in the format 'Patient XXXX Side' or similar
# Let's check if they match our generated keys
metadata_df['match_key'] = metadata_df[id_column].str.strip()

print(f"\nMatch keys from metadata (first 15):")
print(metadata_df['match_key'].head(15).tolist())

## 8. Match and Merge Data

In [ ]:
# Perform the merge/join
print("Matching Stulberg AP data with classification_metadata...\n")

# Create a subset of Stulberg with only relevant columns
stulberg_to_merge = stulberg_ap_df[['match_key', 'Alex']].copy()
stulberg_to_merge.columns = ['match_key', 'stulberg_label']
stulberg_to_merge = stulberg_to_merge[stulberg_to_merge['match_key'].notna()]

print(f"Stulberg AP data to merge:")
print(f"  Rows: {len(stulberg_to_merge)}")
print(stulberg_to_merge.head(10))

print(f"\nMetadata match keys:")
print(f"  Rows: {len(metadata_df)}")
print(metadata_df[['match_key']].head(10))

# Inner join to keep only matching entries
merged_df = metadata_df.merge(
    stulberg_to_merge,
    on='match_key',
    how='inner'
)

print(f"\nMerge results:")
print(f"  Original metadata rows: {len(metadata_df)}")
print(f"  Stulberg AP rows: {len(stulberg_to_merge)}")
print(f"  Matched rows: {len(merged_df)}")

print(f"\nMerged data (first 10 rows):")
print(merged_df.head(10))

## 9. Create Updated Classification Metadata

In [ ]:
# Prepare output dataframe in the same format as original
print("Preparing output file...\n")

# Get all original columns from metadata
original_columns = [col for col in metadata_df.columns if col != 'match_key']
print(f"Original columns: {original_columns}")

# Select output columns
output_df = merged_df[original_columns].copy()

print(f"\nOutput dataframe shape: {output_df.shape}")
print(f"\nOutput dataframe (first 15 rows):")
print(output_df.head(15))

print(f"\nOutput dataframe columns:")
print(output_df.columns.tolist())

print(f"\nOutput statistics:")
print(output_df.describe())

## 10. Export to Excel

In [ ]:
# Save to Excel
print("Exporting to classification_metadata2.xlsx...\n")

# Save with the same sheet name as original
output_df.to_excel(OUTPUT_FILE, sheet_name='samples', index=False)

print(f"Successfully saved to: {OUTPUT_FILE}")
print(f"\nFile statistics:")
print(f"  Rows: {len(output_df)}")
print(f"  Columns: {len(output_df.columns)}")

# Verify the output
verification_df = pd.read_excel(OUTPUT_FILE, sheet_name='samples')
print(f"\nVerification - Read back from file:")
print(f"  Shape: {verification_df.shape}")
print(f"\nFirst 10 rows:")
print(verification_df.head(10))

## 11. Summary and Statistics

In [ ]:
print("\n" + "="*70)
print("LABEL UPDATE SUMMARY")
print("="*70)

print(f"\n1. INPUT DATA")
print(f"   - Stulberg classification file: {STULBERG_FILE.name}")
print(f"   - Total rows in Stulberg sheet 2: {len(stulberg_df)}")
print(f"   - AP view images extracted: {len(stulberg_ap_df)}")
print(f"   - Valid match keys created: {stulberg_ap_df['match_key'].notna().sum()}")

print(f"\n2. ORIGINAL METADATA")
print(f"   - Original file: {METADATA_FILE.name}")
print(f"   - Total entries: {len(metadata_df)}")

print(f"\n3. MATCHING RESULTS")
print(f"   - Entries found in both files: {len(output_df)}")
print(f"   - Match percentage: {len(output_df) / len(metadata_df) * 100:.1f}%")

print(f"\n4. OUTPUT FILE")
print(f"   - Filename: {OUTPUT_FILE.name}")
print(f"   - Location: {OUTPUT_FILE}")
print(f"   - Rows: {len(output_df)}")
print(f"   - Columns: {len(output_df.columns)}")

print(f"\n5. COLUMNS IN OUTPUT")
for col in output_df.columns:
    print(f"   - {col}")

print(f"\n" + "="*70)